# TwiZlatan

For today's lesson, we're gonna learn something more practical: Twitter scraping.

The topic will be, as some of you already guessed from the name of the notebook, this guy:

![image](H:/Data science/Git/Python-Book-Club/png/zlatan.png)

Let's start by importing the required packages:

In [1]:
from twython import Twython